In [105]:
#!pip install onnxruntime

In [106]:
import onnx
import onnxruntime
import os
import numpy as np 
import torch
import torch.nn as nn
import torch.optim as opt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.model_selection import train_test_split
import pandas as pd

In [107]:
torch_device = "cpu"

if torch.cuda.is_available():
    torch_device = "cuda:0"

In [108]:
dataframe = pd.read_csv("Concrete_Data.csv")

In [109]:
X_train, X_test, y_train, y_test = train_test_split(dataframe.to_numpy()[:,:-1],dataframe.to_numpy()[:,-1:], test_size=.20, random_state=42)

y_train = y_train.reshape((-1,1))
y_test =  y_test.reshape((-1,1))

print(X_train[0].shape)
print(y_train[0].shape)
print()
print(X_test.shape)
print(y_test.shape)



(8,)
(1,)

(206, 8)
(206, 1)


In [110]:
X_train = torch.tensor(X_train, dtype=torch.long)
y_train = torch.tensor(y_train, dtype=torch.long)

train_set = [ ( X_train[i], y_train[i].item() ) for i in range(X_train.shape[0])]
train_dl = torch.utils.data.DataLoader( train_set, batch_size=16, shuffle=True )

In [111]:
X_test = torch.tensor(X_test, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.long)

test_set = [ ( X_test[i], y_test[i].item() ) for i in range(X_test.shape[0])]
test_dl = torch.utils.data.DataLoader( test_set, batch_size=y_test.shape[0], shuffle=True )

In [112]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(

            nn.Linear(8, 20),
            nn.ReLU(),
            nn.Dropout(.20),

            nn.Linear(20, 5),
            nn.ReLU(),
            nn.Dropout(.20),

            nn.Linear(5, 1),
        )

    def forward(self, x):
        return self.model(x)

In [113]:
model = MLP()
loss_fn = nn.CrossEntropyLoss()
optimizer = opt.Adam(model.parameters(), lr=.001)
epochs = 200

In [114]:
def training_loop( N_Epochs, model, loss_fn, opt ):
    for epoch in range(N_Epochs):
        for xb, yb in train_dl:

            xb = xb.to(torch.float)
            yb = yb.view((-1,1)).to(torch.float)


            y_pred = model(xb)
            #print(yb)
            #print(y_pred)
            #print()

            loss = loss_fn(y_pred, yb)

            opt.zero_grad()
            loss.backward()
            opt.step()

        if (epoch+1) % 5 == 0:
            print()
            print(epoch+1, "loss=", loss)


In [115]:
model.train()
training_loop(epochs, model, loss_fn, optimizer)


5 loss= tensor(-0., grad_fn=<DivBackward1>)

10 loss= tensor(-0., grad_fn=<DivBackward1>)

15 loss= tensor(-0., grad_fn=<DivBackward1>)

20 loss= tensor(-0., grad_fn=<DivBackward1>)

25 loss= tensor(-0., grad_fn=<DivBackward1>)

30 loss= tensor(-0., grad_fn=<DivBackward1>)

35 loss= tensor(-0., grad_fn=<DivBackward1>)

40 loss= tensor(-0., grad_fn=<DivBackward1>)

45 loss= tensor(-0., grad_fn=<DivBackward1>)

50 loss= tensor(-0., grad_fn=<DivBackward1>)

55 loss= tensor(-0., grad_fn=<DivBackward1>)

60 loss= tensor(-0., grad_fn=<DivBackward1>)

65 loss= tensor(-0., grad_fn=<DivBackward1>)

70 loss= tensor(-0., grad_fn=<DivBackward1>)

75 loss= tensor(-0., grad_fn=<DivBackward1>)

80 loss= tensor(-0., grad_fn=<DivBackward1>)

85 loss= tensor(-0., grad_fn=<DivBackward1>)

90 loss= tensor(-0., grad_fn=<DivBackward1>)

95 loss= tensor(-0., grad_fn=<DivBackward1>)

100 loss= tensor(-0., grad_fn=<DivBackward1>)

105 loss= tensor(-0., grad_fn=<DivBackward1>)

110 loss= tensor(-0., grad_fn=<D

In [116]:
dummy_input = torch.rand (1, 8).to(torch_device)
onnx_path = "onnx_model.onnx"

torch.onnx.export(
    model,
    dummy_input,
    onnx_path,
    verbose=False,
    input_names=['input'],
    output_names=['output']
)

print(f"Model exported to {onnx_path}")

Model exported to onnx_model.onnx


In [117]:
#load model
onnx_model = onnx.load(onnx_path)

#create runtime session
ort_session = onnxruntime.InferenceSession(onnx_path)

#fill with dummy data
input_data = dummy_input.cpu().numpy()

#run model with ONNX runtime
ort_inputs={ort_session.get_inputs()[0].name: input_data}
ort_outputs=ort_session.run(None, ort_inputs)
print("ONNX Runtime outputs", ort_outputs)

ONNX Runtime outputs [array([[-0.09379324]], dtype=float32)]
